In [1]:
%load_ext autoreload

In [4]:
%autoreload 

import sys
sys.path.append('../../..')
import pickle

import numpy as np
from tqdm import tqdm_notebook as tqdm

import matplotlib
matplotlib.rc_file('../../matplotlibrc')
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 
import matplotlib.colors as colors
%matplotlib inline

import darkhistory.physics as phys
import main
import config


Bad key text.latex.preview in file ../../matplotlibrc, line 10 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution


In [6]:
config.test(end_rs=200, iter=2, std_only=True)
# config.test()

Running main.evolve(...): 
******************************************
Testing solution with no DM: 


  0%|          | 0/170.0 [00:00<?, ?it/s]

  0%|          | 0/170.0 [00:00<?, ?it/s]

The maximum relative change in xHI and xHeI is:  1.128741189826421e-05
The maximum relative change in Tm is:  5.27343657630297e-08
The maximum relative change in f_(H ion) is:  1.406972867989964e-06
The maximum relative change in f_(H ion) is:  1.406972867989964e-06
The maximum relative change in f_(He ion) is:  1.0759358923806417e-06
The maximum relative change in f_(Lya) is:  1.8421043737060216e-06
The maximum relative change in f_(heat) is:  3.008904892132236e-06
The maximum relative change in f_(cont) is:  6.611322835858857e-07
The maximum relative change in the MLA parameters is:  1.3401875926355444e-05
Pickled solution with no DM!
Test complete!


In [ ]:
test_data = pickle.load(open('/scratch/gpfs/hongwanl/DarkHistory/data/std_test_data.p', 'rb'))

ref_data = pickle.load(open('/scratch/gpfs/hongwanl/DarkHistory/data/reference_20220818_std_result_n_10_high_rs_1555_coarsen_16_reion_False_rtol_1e-6.p', 'rb'))

In [ ]:
print(test_data['MLA'][1:])
print(ref_data['MLA'][1:])

In [20]:
options_dict = {
    'primary':'elec_delta', 'DM_process':'decay', 'mDM':1e8, 'lifetime':3e25,
    'start_rs': 3000, 'high_rs': 1.555e3, 'end_rs':4,
    'reion_switch':True, 'reion_method':'Puchwein', 'heat_switch':True,
    'coarsen_factor':16, 'distort':True, 'fexc_switch': True, 
    'MLA_funcs':None,
    'reprocess_distortion':True, 'nmax':10, 'rtol':1e-6, 'use_tqdm':True, 'iterations':2
}

options_dict_std = {
    'primary':'elec_delta', 'DM_process':'decay', 'mDM':1e8, 'lifetime':3e40,
    'start_rs': 3000, 'high_rs': 1.555e3, 'end_rs':2000,
    'reion_switch':False, 'reion_method':'Puchwein', 'heat_switch':True,
    'coarsen_factor':16, 'distort':True, 'fexc_switch': True, 
    'MLA_funcs':None,
    'reprocess_distortion':True, 'nmax':10, 'rtol':1e-6, 'use_tqdm':True, 'iterations':2
}

In [33]:
# %%prun 
# res = main.evolve(**options_dict)
# res_std = main.evolve(**options_dict_std)
%lprun -m darkhistory.low_energy.atomic main.evolve(**options_dict_std)


UsageError: Could not find module low_energy.atomic.process_MLA.
NameError: name 'low_energy' is not defined


In [ ]:
plt.figure()

plt.loglog()

res_rs = res[-1]['MLA'][0]
x_std = phys.x_std(res_rs)
Tm_std = phys.Tm_std(res_rs)

C_times_alpha_B = phys.peebles_C(x_std, res_rs) * phys.alpha_recomb(Tm_std, 'HI')


# plt.plot(res['MLA'][0], res['MLA'][1,:], label=r'DM $\tilde{\alpha}_\mathrm{B} / (C_\mathrm{std} \alpha_{B,\mathrm{std}})$')
plt.plot(res_std[-1]['MLA'][0], res_std[-1]['MLA'][1,:] / C_times_alpha_B, label=r'Standard $\tilde{\alpha}_\mathrm{B} / (C_\mathrm{std} \alpha_{B, \mathrm{std}})$')
# plt.plot(res_std_2['MLA'][0], res_std_2['MLA'][1,:] / C_times_alpha_B, label=r'Standard $\tilde{\alpha}_\mathrm{B} / (C_\mathrm{std} \alpha_{B, \mathrm{std}})$')
# plt.plot(res_std['MLA'][0], res_std['MLA'][1,:] / res_std_2['MLA'][1,:])
# plt.plot(res_rs, C_times_alpha_B, 'k--', label=r'$C_\mathrm{std} \alpha_{B, \mathrm{std}}$')


plt.xlabel(r'$1+z$')
plt.ylabel(r'Rate [cm$^{3}$ s$^{-1}$]')
plt.legend()

# plt.xlim(1e3, 3e3)

In [ ]:
plt.figure()

plt.loglog()

plt.plot(res_std['rs'], res_std['x'][:,0] / phys.x_std(res_std['rs']))

plt.plot(res['rs'], res['x'][:,0] / phys.x_std(res['rs']))


# plt.xlim(20, 3000)
# plt.ylim(0.975, 1.025)

In [ ]:
pickle.dump(res_std, open('/scratch/gpfs/hongwanl/DarkHistory/data/reference_20220822_std_result_n_10_high_rs_1555_coarsen_16_reion_False_rtol_1e-6_iter_2.p', 'wb'))

pickle.dump(res, open('/scratch/gpfs/hongwanl/DarkHistory/data/reference_20220822_mDM_1e8_elec_delta_decay_3e25_n_10_high_rs_1555_coarsen_16_reion_True_rtol_1e-6_iter_2.p', 'wb'))


In [ ]:
nmax = 10

eng_norm_levels = np.divide(
        1., np.arange(nmax+1.)**2, out=np.ones(nmax+1)*np.nan, 
        where=np.arange(nmax+1) != 0
    )
Ennp_mat = (eng_norm_levels[None,:] - eng_norm_levels[:,None]) 

In [ ]:
Ennp_mat[1]

In [ ]:

import darkhistory.low_energy.atomic as atomic

nmax = 50

R = atomic.populate_radial(nmax)

BB, BB_2s1s = atomic.populate_bound_bound(nmax, 0.5, R, Delta_f=None, simple_2s1s=True)

# Initialize K_ij = R_ji / R_i,tot as n x n' x l x l', where i = (n,l), j = (n',l')
K = np.zeros((nmax+1, nmax+1, nmax+1, nmax+1))
# BB['up'] has dimensions n x n' x l, with transitions to l' = l+1. 
K_up = np.zeros_like(K) + BB['up'][:,:,:,None]
# print(K_up)
# print(K_up[5,3])
K_up = np.tril(np.triu(K_up, k=1), k=1)
# print(K_up[5,3])
# print(K_up)
# BB['dn'] has dimensions n x n' x l, with transitions to l' = l-1. e
K_dn = np.zeros_like(K) + BB['dn'][:,:,:,None]
K_dn = np.tril(np.triu(K_dn, k=-1), k=-1)


K = K_up + K_dn 


In [ ]:
# i = n(n+1)/2 - n + l
# j = np(np+1)/2 - np + lp

num_states = int(nmax*(nmax+1)/2)

K_2d = np.zeros((num_states, num_states))

K_2d_new = np.zeros((num_states, num_states))

states_n = np.concatenate([
    list(map(int, k*np.ones(k))) for k in range(1, nmax+1, 1)]
)
states_l = np.concatenate([np.arange(k) for k in range(1, nmax+1)])

for i in np.arange(num_states): 
    for j in np.arange(num_states): 
        K_2d[i,j] = K[states_n[i], states_l[i], states_n[j], states_l[j]]


In [ ]:
states_n_p_2d = np.tile(states_n, (num_states,1)) 
states_l_p_2d = np.tile(states_l, (num_states,1)) 
states_n_2d = np.transpose(states_n_p_2d)
states_l_2d = np.transpose(states_l_p_2d)

K_2d_new = K[states_n_2d, states_l_2d, states_n_p_2d, states_l_p_2d]

In [ ]:
# print(K_2d)
print(K_2d_new.shape)

In [ ]:
print(np.array_equal(K_2d_new, K_2d))

In [ ]:
np.arange(2, nmax+1,1)

In [10]:
%load_ext line_profiler
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat